<a href="https://colab.research.google.com/github/AbdullahFarg/Assignment_1/blob/main/20210053_20210541.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# "~(for_all(x)[for_all(y)[for_all(z)[imply(imply(P(y),Q(z)),imply(P(x),Q(x)))]]])"


# dic = {
#     "not":
#      {
#          "for_all(x)":
#           {
#               "for_all(y)":
#                {
#                    "for_all(z)":
#                    {
#                        "imply":
#                         (
#                             {"imply": (P(y),Q(z))},
#                             {"imply": (P(x),Q(x))}
#                         )
#                    }
#                }
#           }
#       }
#   }

In [2]:
def __split (statement) :
    statement.replace( " " , "")
    comp = []
    quantifier, body = statement.split("(", 1)
    body = body[ : -1]
    body += ','
    cur = ""
    c = 0
    for ch in body :
      if(ch == '(') :
        c+=1
      elif(ch == ')') :
        c-=1
      if(ch == ',' and c == 0) :
        comp.append(cur)
        cur = ""
      if(c== 0 and ch in [" " , ","]) :
        continue
      cur+= ch
    return comp

expression = "im(im(P(y) , Q(z)) , im(P(x) , Q(x)))"

print(__split(expression))


['im(P(y) , Q(z))', 'im(P(x) , Q(x))']


In [3]:
def parse_statement(statement):
    # Base case: If the statement starts with "~", it's a negation
    if statement.startswith("not"):
        quantifier, body = statement.split("(", 1)
        body = body[:-1]
        return {"not": parse_statement(body)}

    # Base case: If the statement starts with a universal quantification (for_all)
    if statement.startswith("for_all"):
        quantifier, body = statement.split("[", 1)
        body = body[:-1]
        return {quantifier: parse_statement(body)}

    # Base case: If the statement starts with an existential quantification (there_exist)
    if statement.startswith("there_exist"):
        quantifier, body = statement.split("[", 1)
        body = body[:-1]
        return {quantifier: parse_statement(body)}

    # Base case: If the statement starts with a conjunction (and)
    if statement.startswith("and"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"and": comps_parsed}

    # Base case: If the statement starts with a disjunction (or)
    if statement.startswith("or"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"or": comps_parsed}

    # Base case: If the statement starts with a implecation (imply)
    if statement.startswith("imply"):
        comps = __split(statement)
        comps_parsed = []
        for el in comps:
            comps_parsed.append(parse_statement(el))

        return {"imply": comps_parsed}

    return statement


# -------------------------------------------------- Test:1
print("Test1:")
statement = "for_all(x)[z]"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
print("---------------------------------------")

# -------------------------------------------------- Test:2
print("Test2:")
statement = "and(or(A, B), C, imply(X, Z))"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
print("---------------------------------------")

# -------------------------------------------------- Test:3
print("Test3:")
statement = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
print("---------------------------------------")

Test1:
Original statement: for_all(x)[z]

Parsed statement: {'for_all(x)': 'z'}

---------------------------------------
Test2:
Original statement: and(or(A, B), C, imply(X, Z))

Parsed statement: {'and': [{'or': ['A', 'B']}, 'C', {'imply': ['X', 'Z']}]}

---------------------------------------
Test3:
Original statement: not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])

Parsed statement: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}

---------------------------------------


In [4]:
def Eliminate_implication(my_dict):
    # Base case: If reached the max depth (no more dictionaries or lists)
    if not isinstance(my_dict, dict):
        return my_dict

    # Get the key and the value of the current dictionary
    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    # Apply Eliminate the Implication for all the values recursively
    if isinstance(first_value, list):
      for val in first_value:
          Eliminate_implication(val)
    elif isinstance(first_value, dict):
        Eliminate_implication(first_value)

    # Eliminate the current implication
    if "imply" in my_dict:
      my_dict['or'] = my_dict.pop('imply')
      my_dict['or'][0] = {'not': my_dict['or'][0]}

# -------------------------------------------------- Test:1
print("Test1:")
statement = "imply(A, B)"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
print("---------------------------------------")

# -------------------------------------------------- Test:2
print("Test2:")
statement = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
print("---------------------------------------")

Test1:
Original statement: imply(A, B)

Parsed statement: {'imply': ['A', 'B']}

After Eliminate Implication: {'or': [{'not': 'A'}, 'B']}

---------------------------------------
Test2:
Original statement: not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])

Parsed statement: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}

After Eliminate Implication: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'or': [{'not': {'or': [{'not': 'P(y)'}, 'Q(z)']}}, {'or': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}

---------------------------------------


In [5]:
# not (a or b) = (not a and not b)
# not (a and b) = (not a or not b)
# not forall() = therE not()
# not thereE() = forall not()

def Move_negation_inward(my_dict):
    # Base case1: If reached the max depth (no more dictionaries or lists)
    if not isinstance(my_dict, dict):
          return my_dict

    # Get the key and the value of the current dictionary
    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    # If the current key is not, then move it inward
    if first_key == "not":
      # Base case2: If the current value is string, do nothing
      if isinstance(first_value, str):
        return

      # Base case4: if the current value is dictionary, apply the not the it's value recursively
      if isinstance(first_value, dict):
        keys = iter(first_value)
        key = next(keys)
        value = first_value[key]

        if key == "not":
            Move_negation_inward(value)
            return

        if key == "and":
            # not(and[a, b]) ==> or(not([a, b])) ==>
            # not(not(val)) ==> val
            my_dict["or"] = my_dict.pop("not")
            my_dict["or"] = []
            for val in value:
                val = {"not": val}
                my_dict["or"].append(val)
                Move_negation_inward(val)
            return

        elif key == "or":
            my_dict["and"] = my_dict.pop("not")
            my_dict["and"] = []
            for val in value:
                val = {"not": val}
                my_dict["and"].append(val)
                Move_negation_inward(val)
            return

        elif key.startswith("for_all"):
            k = "there_exist(" + str(key.split("(", 1)[1][:-1]) + ")"
            my_dict[k] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        elif key.startswith("there_exist"):
            k = "for_all(" + str(key.split("(", 1)[1][:-1]) + ")"
            my_dict[k] = my_dict.pop("not")
            first_value["not"] = first_value.pop(key)

        Move_negation_inward(first_value)
        return

    else:
      if isinstance(first_value, list):
        for val in first_value:
          Move_negation_inward(val)
      elif isinstance(first_value, dict):
        Move_negation_inward(first_value)


# -------------------------------------------------- Test:1
print("Test1:")
statement = "not(and(A, B))"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
print("---------------------------------------")

# -------------------------------------------------- Test:2
print("Test2:")
statement = "imply(and(A, B), C)"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
print("---------------------------------------")

# -------------------------------------------------- Test:3
print("Test3:")
statement = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
print("---------------------------------------")

Test1:
Original statement: not(and(A, B))

Parsed statement: {'not': {'and': ['A', 'B']}}

After Eliminate Implication: {'not': {'and': ['A', 'B']}}

After Move negation inward: {'or': [{'not': 'A'}, {'not': 'B'}]}
---------------------------------------
Test2:
Original statement: imply(and(A, B), C)

Parsed statement: {'imply': [{'and': ['A', 'B']}, 'C']}

After Eliminate Implication: {'or': [{'not': {'and': ['A', 'B']}}, 'C']}

After Move negation inward: {'or': [{'or': [{'not': 'A'}, {'not': 'B'}]}, 'C']}
---------------------------------------
Test3:
Original statement: not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])

Parsed statement: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}

After Eliminate Implication: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'or': [{'not': {'or': [{'not': 'P(y)'}, 'Q(z)']}}, {'or': [{'not': 'P(x)'}, 'Q(x)']}]}}}}}



In [6]:
def Remove_double_not(my_dict):
    while True:
        # Base case1: If reached the max depth (no more dictionaries or lists)
        if not isinstance(my_dict, dict):
              return my_dict

        # Get the key and the value of the current dictionary
        keys_iterator = iter(my_dict)
        first_key = next(keys_iterator)
        first_value = my_dict[first_key]

        # discard the unedded nots in the first
        if first_key == "not" and isinstance(first_value, dict):

            # not(not(and(A, B))) == > and(A, B)

            # Get the key and the value of the second dictionary
            keys = iter(first_value)
            key = next(keys)
            value = first_value[key]

            # Discard the extra nots
            my_dict = value

        else:
            break

    return my_dict

def Remove_double_not_rec(my_dict):
    # Base case1: If reached the max depth (no more dictionaries or lists)
    if not isinstance(my_dict, dict):
          return my_dict

    # Get the key and the value of the current dictionary
    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    if isinstance(first_value, list):
        for i in range(len(first_value)):
            my_dict[first_key][i] = Remove_double_not(first_value[i])
            Remove_double_not_rec(first_value[i])
        return

    # Check for double not
    if isinstance(first_value, dict):
        # Get the key and the value of the second dictionary
        keys2 = iter(first_value)
        key2 = next(keys2)
        value2 = first_value[key2]

        if isinstance(value2, dict):
            # Get the key and the value of the second dictionary
            keys3 = iter(value2)
            key3 = next(keys3)
            value3 = value2[key3]

            if key2 == "not" and key3 == "not":
                my_dict[first_key] = value3

        Remove_double_not_rec(first_value)


# -------------------------------------------------- Test:1
print("Test1:")
statement = "not(not(A))"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
my_dict = Remove_double_not(my_dict)
print(f"After removing double nots (phase1): {my_dict}")
print("---------------------------------------")

# -------------------------------------------------- Test:2
print("Test2:")
statement = "not(for_all(x)[not(or(not(not(C)), X))])"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
my_dict = Remove_double_not(my_dict)
Remove_double_not_rec(my_dict)
print(f"After removing double nots: {my_dict}")
print("---------------------------------------")

# -------------------------------------------------- Test:3
print("Test3:")
statement = "not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])"
print(f"Original statement: {statement}\n")
my_dict =  parse_statement(statement)
print(f"Parsed statement: {my_dict}\n")
Eliminate_implication(my_dict)
print(f"After Eliminate Implication: {my_dict}\n")
Move_negation_inward(my_dict)
print(f"After Move negation inward: {my_dict}")
my_dict = Remove_double_not(my_dict)
Remove_double_not_rec(my_dict)
print(f"After removing double nots: {my_dict}")
print("---------------------------------------")

Test1:
Original statement: not(not(A))

Parsed statement: {'not': {'not': 'A'}}

After Eliminate Implication: {'not': {'not': 'A'}}

After Move negation inward: {'not': {'not': 'A'}}
After removing double nots (phase1): A
---------------------------------------
Test2:
Original statement: not(for_all(x)[not(or(not(not(C)), X))])

Parsed statement: {'not': {'for_all(x)': {'not': {'or': [{'not': {'not': 'C'}}, 'X']}}}}

After Eliminate Implication: {'not': {'for_all(x)': {'not': {'or': [{'not': {'not': 'C'}}, 'X']}}}}

After Move negation inward: {'there_exist(x)': {'not': {'not': {'or': [{'not': {'not': 'C'}}, 'X']}}}}
After removing double nots: {'there_exist(x)': {'or': ['C', 'X']}}
---------------------------------------
Test3:
Original statement: not(there_exist(x)[for_all(y)[for_all(z)[imply(imply(P(y) ,Q(z)) ,imply(P(x) ,Q(x)))]]])

Parsed statement: {'not': {'there_exist(x)': {'for_all(y)': {'for_all(z)': {'imply': [{'imply': ['P(y)', 'Q(z)']}, {'imply': ['P(x)', 'Q(x)']}]}}}}}

A

In [7]:
# or(for_all(x)[P(X)], there_exist(x)[Q(x)])

variables = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                       'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                       'w', 'x', 'y', 'z']

taken_variables = []

def change_variable_names(new_var, values_to_change):
    if isinstance(values_to_change, str):
        values_to_change = values_to_change.split("(", 1) + "(" + new_var + ")"
        return values_to_change

    if isinstance(values_to_change, list):
        for i in range(len(values_to_change)):
            values_to_change[i] = change_variable_names(new_var, values_to_change[i])
        return values_to_change

    if isinstance(values_to_change, dict):
        # Get the key and the value of the current dictionary
        keys_iterator = iter(values_to_change)
        first_key = next(keys_iterator)
        first_value = values_to_change[first_key]

        if first_key.startswith("for_all") or first_key.startswith("there_exist"):
              return

        values_to_change[first_key] = change_variable_names(first_value)

def Standardize_variable_scope(my_dict):
    # Base case1: If reached the max depth (no more dictionaries or lists)
    if not isinstance(my_dict, dict):
          return my_dict

    # Get the key and the value of the current dictionary
    keys_iterator = iter(my_dict)
    first_key = next(keys_iterator)
    first_value = my_dict[first_key]

    if isinstance(first_value, list):
        for i in range(len(first_value)):
            if isinstance(first_value[i], dict):
                keys = iter(first_value[i])
                key = next(keys)
                value = first_value[i][key]
                if key.startswith("for_all") or key.startswith("there_exist"):
                    variable = key.split("(", 1)[1][:-1]
                    if variable in taken_variables:
                        new_variable = variables[0]
                        my_dict[first_key][i] = change_variable_names(new_variable, first_value[i])
                        print(my_dict[first_key][i])
                        taken_variables.append(new_variable)
                        variables.remove(new_variable)
                        print(my_dict)
                    else:
                        taken_variables.append(variable)
                        variables.remove(variable)
                        print(my_dict)

            Standardize_variable_scope(my_dict[first_key][i])
    else:
        Standardize_variable_scope(my_dict[first_key])

statement = "or(for_all(x)[P(x)], there_exist(x)[Q(x)])"
my_dict = parse_statement(statement)
print(my_dict)
Standardize_variable_scope(my_dict)
print(my_dict)

{'or': [{'for_all(x)': 'P(x)'}, {'there_exist(x)': 'Q(x)'}]}
{'or': [{'for_all(x)': 'P(x)'}, {'there_exist(x)': 'Q(x)'}]}
None
{'or': [{'for_all(x)': 'P(x)'}, None]}
{'or': [{'for_all(x)': 'P(x)'}, None]}
